In [5]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [6]:
# Load and preprocess data
data = pd.read_csv('sonar.csv', header=None)
X = data.iloc[:, :-1].values
y = (data.iloc[:, -1] == 'R').astype(int).values  # Convert R/M to 1/0

In [7]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
# Build model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [11]:
# Train model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.5093 - loss: 0.7683 - val_accuracy: 0.6471 - val_loss: 0.6762
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5609 - loss: 0.6920 - val_accuracy: 0.6765 - val_loss: 0.6499
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7337 - loss: 0.6138 - val_accuracy: 0.7353 - val_loss: 0.6333
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6813 - loss: 0.6109 - val_accuracy: 0.7353 - val_loss: 0.6118
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8120 - loss: 0.5083 - val_accuracy: 0.7353 - val_loss: 0.5858
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8026 - loss: 0.5166 - val_accuracy: 0.7647 - val_loss: 0.5602
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7528 - loss: 0.4995 - val_accuracy: 0.7941 - val_loss: 0.5338
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8336 - loss: 0.4480 - val_accuracy: 0.7941 - val_loss:

In [12]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Test Accuracy: 0.9048
Test Loss: 0.2823


In [13]:
# Save model
model.save('sonar_model.h5')

In [16]:
import gradio as gr
import numpy as np
import tensorflow
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the trained model
model = load_model('sonar_model.h5')

# Load data to fit scaler (same as training)
data = pd.read_csv('sonar.csv', header=None)
X = data.iloc[:, :-1].values
scaler = StandardScaler().fit(X)

# Prediction function
def predict_sonar(*inputs):
    # Convert inputs to numpy array
    input_data = np.array(inputs, dtype=float).reshape(1, -1)

    # Scale input
    input_data = scaler.transform(input_data)

    # Make prediction
    prediction = model.predict(input_data, verbose=0)
    probability = prediction[0][0]

    # Return result
    return "Rock" if probability > 0.5 else "Mine", f"Probability: {probability:.4f}"

# Create input components for 60 features
inputs = [gr.Number(label=f"Feature {i+1}") for i in range(60)]

# Create Gradio interface
interface = gr.Interface(
    fn=predict_sonar,
    inputs=inputs,
    outputs=[gr.Text(label="Prediction"), gr.Text(label="Probability")],
    title="Sonar Rock vs Mine Prediction",
    description="Enter 60 feature values to predict if the sonar signal is from a Rock or Mine."
)

# Launch interface
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e0caa2684a736a2e5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
